# Exercise 0

In [1]:
import psycopg2

pg_connection_dict = {
    'dbname': 'GeoDB',
    'user': 'postgres',
    'password': '1234',
    'port': 5432,
    'host': 'localhost'
}

connection = psycopg2.connect(**pg_connection_dict)
cursor = connection.cursor()

# Q1:
#### Create a new database called vmarkets and activate/select it

In [2]:
connection.autocommit = True
cursor.execute("CREATE DATABASE vmarkets;")
connection.autocommit = False
connection.close()
pg_connection_dict['dbname'] = 'vmarkets'
connection = psycopg2.connect(**pg_connection_dict)
cursor = connection.cursor()

# Q2
#### In vmarkets, create the following table/relation via SQL (in pgAdmin, the query tool): vegetables(v_id, v_type)

In [4]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS vegetables (
        v_id INT,
        v_type TEXT,
        CONSTRAINT vegetables_pk PRIMARY KEY (v_id)
    );
""")
connection.commit()

# Q3
#### Insert the tuples (’Tomatos’, 1) and (’Salad’, 2) into vegetables.

In [5]:
data = [
    ("Tomatos", 1),
    ("Salad", 2)
]
cursor.executemany("""
    INSERT INTO vegetables (v_type, v_id)
    VALUES (%s, %s);
""", data)

connection.commit()

# Q4
#### Now, why can’t you insert the tuple (’Taters’, 2)? Make the necessary adjustment to insert ’Taters’.

In [6]:
cursor.execute("""
    INSERT INTO vegetables (v_type, v_id)
    VALUES ('Taters', 3);
""")
connection.commit()

# Q5
#### Create a file additional_vegetables.csv and write down a valid tuple in .csv format, e.g., Cucumbers, 4. If you are using docker, make sure to locate the file in an accessible directory specified in the docker-compose.yml (e.g., external_data, as discussed).

In [7]:
data = [
    ['Cucumbers', 4],
    ['Potato', 5],
    ['Onion', 6]
]

with open("additional_vegetables.csv", "w") as file:
    for item in data:
        file.write(f"{item[0]}, {item[1]}\n")

# Q6
#### Use an SQL query to insert the data from the .csv file into the table vegetables.


In [8]:
cursor.execute("""
    COPY vegetables(v_type, v_id)
    FROM '/Users/abdulraheem/Documents/GeoDB/additional_vegetables.csv'
    WITH (FORMAT csv, HEADER false)
""")
connection.commit()

# Q7
#### Create a file markets.csv with the content:
##### market_name, market_id
##### rewe, 1
##### tegut, 2

In [9]:
with open("markets.csv", "w") as file:
    data = [
        ('market_name', 'market_id'),
        ('rewe', 1),
        ('tegut', 2)
    ]
    for item in data:
        file.write(f"{item[0]}, {item[1]}\n")

# Q8
#### Create a foreign table markets_csv with direct access to markets.csv 

In [10]:
ENABLE_EXTENSION = "CREATE EXTENSION IF NOT EXISTS file_fdw;"
CREATE_FOREIGN_SERVER = "CREATE SERVER csv_server FOREIGN DATA WRAPPER file_fdw;"
CREATE_FOREIGN_TABLE = """
    CREATE FOREIGN TABLE markets_csv (
        market_name TEXT,
        market_id INT
    )
    SERVER csv_server
    OPTIONS (filename '/Users/abdulraheem/Documents/GeoDB/markets.csv', format 'csv', header 'true');
"""
cursor.execute(ENABLE_EXTENSION)
cursor.execute(CREATE_FOREIGN_SERVER)
cursor.execute(CREATE_FOREIGN_TABLE)
connection.commit()

# Q9
#### Add a column m_id to the table vegetables.

In [ ]:
ALTER_TABLE = """
    ALTER TABLE vegetables
    ADD COLUMN m_id INT;
"""
cursor.execute(ALTER_TABLE)
connection.commit()

# Q10
#### Set vegetables.m_id to 1 for some tuples and 2 for the others.

In [13]:
GET_ALL_VEGGIES = "SELECT * FROM vegetables"
UPDATE_QUERY = "UPDATE vegetables SET m_id = {} WHERE v_id = {}"
cursor.execute(GET_ALL_VEGGIES)
veggeis = cursor.fetchall()
for n in range(len(veggeis) + 1):
    cursor.execute(UPDATE_QUERY.format(n%2 + 1, n))
connection.commit()

# Q11
#### We want to add a foreign key constraint to vegetables. Why does
###### ALTER TABLE vegetables
###### ADD CONSTRAINT fk_veggies_markets
###### FOREIGN KEY ( m_id )
###### REFERENCES markets ( m_id ) ;
#### not work?

Because markets does not exist

# Q12
#### Create a table markets with the content of markets_csv.

In [14]:
CREATE_MARKETS_TABLE = """
    CREATE TABLE markets AS
    SELECT *
    FROM markets_csv;
"""
cursor.execute(CREATE_MARKETS_TABLE)
connection.commit()

# Q13
#### Now, why does
###### ALTER TABLE vegetables
###### ADD CONSTRAINT fk_veggies_markets
###### FOREIGN KEY ( m_id )
###### REFERENCES markets ( m_id ) ;
#### still not work?

In [19]:
ALTER_TABLE = """
    ALTER TABLE vegetables
    ADD CONSTRAINT fk_veggies_markets
    FOREIGN KEY ( m_id )
    REFERENCES markets ( market_id ) ;
"""
cursor.execute(ALTER_TABLE)
connection.commit()

InvalidForeignKey: there is no unique constraint matching given keys for referenced table "markets"

# Q14
#### Solve the problem by altering the table markets accordingly and add the foreign key referencing markets.m_id to vegetables

In [18]:
ALTER_TABLE = """
    ALTER TABLE markets
    ADD CONSTRAINT markets_pk PRIMARY KEY (market_id);
"""
cursor.execute(ALTER_TABLE)
connection.commit()

# Q15
#### Output the names of all markets and the number of vegetables present at each market (use the GROUP BY clause and an aggregate function)

In [20]:
GET_ALL_MARKETS = """
    SELECT m.market_name, COUNT(v.v_id) as vegetable_count
    FROM markets m
    LEFT JOIN vegetables v
        ON v.m_id = m.market_id
    GROUP BY m.market_name
"""
cursor.execute(GET_ALL_MARKETS)
markets = cursor.fetchall()
print(markets)

[('tegut', 3), ('rewe', 3)]


# Q16
#### Print only those markets having at least 2 vegetables.

In [21]:
GET_ALL_MARKETS_2 = """
    SELECT m.market_name, COUNT(v.v_id) as vegetable_count
    FROM markets m
    LEFT JOIN vegetables v
        ON v.m_id = m.market_id
    GROUP BY m.market_name
    HAVING COUNT(v.v_id) >= 2
"""
cursor.execute(GET_ALL_MARKETS_2)
markets = cursor.fetchall()
print(markets)

[('tegut', 3), ('rewe', 3)]


# Q17
#### Create a new type called point2d, where a point consist of an id, an x-coordinate and a y-coordinate.

In [34]:
CREATE_POINT_TYPE = """
    CREATE TYPE point2d AS (
        id INT,
        x DOUBLE PRECISION,
        y DOUBLE PRECISION
    )
"""
cursor.execute(CREATE_POINT_TYPE)
connection.commit()

# Q18
#### Extend the table markets to include a column loc of type point2d. Update the table such that each market has a valid location of type point2d.

In [41]:
ALTER_TABLE = """
    ALTER TABLE markets
    ADD COLUMN loc point2d
"""
cursor.execute(ALTER_TABLE)

UPDATE_QUERY = """
    UPDATE markets
    SET loc = ({}, {}, {})
"""
update_data = [
    {'id': 1, 'x': 2.12, 'y': 3.22},
    {'id': 2, 'x': 5.12, 'y': 4.22}
]
for loc in update_data:
    cursor.execute(UPDATE_QUERY.format(loc['id'], loc['x'], loc['y']))
connection.commit()

# Q19
#### Extra: Write a function called euclidean_distance that returns the euclidean distance of 2 points.

In [48]:
CREATE_FUNCTION = """
    CREATE OR REPLACE FUNCTION euclidean_distance (p1 point2d, p2 point2d)
    RETURNS DOUBLE PRECISION AS $$
    BEGIN
        RETURN sqrt(power(p1.x - p2.x, 2) + power(p1.y - p2.y, 2));
    END;
    $$ LANGUAGE plpgsql IMMUTABLE;
"""
cursor.execute(CREATE_FUNCTION)
connection.commit()